This example shows how to train a supervised learning algorithm for classification using one of these TextClassification datasets.

In [1]:
import torch
import torchtext as t_text
from torchtext.datasets import text_classification

import os

import torch.nn as nn
import torch.nn.functional as fxnl

from torch.utils.data import DataLoader

import time 
from torch.utils.data.dataset import random_split


import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

In [2]:
NGRAMS = 2 

if not os.path.isdir('./.dataset'):
    os.mkdir('./.dataset')

train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](root='./.dataset', ngrams=NGRAMS, vocab=None)

BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

120000lines [00:11, 10251.17lines/s]
120000lines [00:22, 5375.93lines/s]
7600lines [00:01, 4938.82lines/s]


In [3]:
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim,num_classes):
        super(TextSentiment, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_classes)
        self.init_weights()
    
    def init_weights(self):
        init_range = 0.5
        self.embedding.weight.data.uniform_(-init_range, init_range)
        self.fc.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()
    
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [4]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(vocab_size=VOCAB_SIZE, embed_dim=EMBED_DIM, num_classes=NUN_CLASS).to(device)

In [5]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)

    return text, offsets, label

In [6]:
def train_fxn(sub_train_):
    train_loss = 0 
    train_accuracy = 0

    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)

    for idx, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()

        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)

        loss = loss_fxn(output, cls)
        train_loss += loss.item()
        loss.backward()

        optimizer.step()

        train_accuracy += (output.argmax(1) == cls).sum().item()
    
    scheduler.step()

    return train_loss / len(sub_train_), train_accuracy / len(sub_train_)


In [7]:
def test(data_):
    loss = 0
    accuracy = 0

    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)

    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)

        with torch.no_grad():
            output = model(text, offsets)
            
            loss_ = loss_fxn(output, cls)
            loss +=loss_.item()
            accuracy += (output.argmax(1) == cls).sum().item()
    
    return loss / len(data_), accuracy / len(data_)


In [8]:
N_EPOCHS = 5 
min_valid_loss = float('inf')

loss_fxn = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)


train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_accuracy = train_fxn(sub_train_)
    valid_loss, valid_accuracy = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_accuracy * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_accuracy * 100:.1f}%(valid)')


Epoch: 1  | time in 0 minutes, 36 seconds
	Loss: 0.0261(train)	|	Acc: 84.7%(train)
	Loss: 0.0178(valid)	|	Acc: 90.6%(valid)
Epoch: 2  | time in 0 minutes, 45 seconds
	Loss: 0.0119(train)	|	Acc: 93.7%(train)
	Loss: 0.0168(valid)	|	Acc: 91.5%(valid)
Epoch: 3  | time in 0 minutes, 41 seconds
	Loss: 0.0069(train)	|	Acc: 96.4%(train)
	Loss: 0.0195(valid)	|	Acc: 90.9%(valid)
Epoch: 4  | time in 0 minutes, 34 seconds
	Loss: 0.0038(train)	|	Acc: 98.1%(train)
	Loss: 0.0195(valid)	|	Acc: 91.9%(valid)
Epoch: 5  | time in 0 minutes, 31 seconds
	Loss: 0.0023(train)	|	Acc: 99.0%(train)
	Loss: 0.0208(valid)	|	Acc: 91.8%(valid)


In [9]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0259(test)	|	Acc: 89.9%(test)


In [10]:
ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}


In [11]:
def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token] for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1


In [12]:
ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

In [13]:
vocab = train_dataset.get_vocab()
model = model.to("cpu")


In [14]:
print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 2)])

This is a Sports news
